In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

In [32]:
from problems import *

In [33]:
import os
from functools import partial
from torch.utils.data import DataLoader

In [34]:
from scripts.create_problem import load_problem_set
import problems as pbs
from dpt.data import *
from dpt.reward import Reward
from train_dpt import *
from notebooks.utils import *

In [36]:
problem = 'Normal(0, 1)'
data_dir = '../data/normal'
results_dir = '../results/normal'
suffix = 'test'

problems = load_problem_set(os.path.join(data_dir, problem, suffix))
dataset = OfflineDataset(problems, seq_len=100, results_dir=results_dir, suffix=suffix, ad_ratio=0.0, action='point', target_action='gt')
# dataset = OnlineDataset(problems)
sample = dataset[0]

for k, v in sample.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

query_state torch.float32 torch.Size([51])
target_state torch.float32 torch.Size([51])
problem <class 'problems.qubo.Distribution'>
states torch.float32 torch.Size([100, 51])
actions torch.int64 torch.Size([100, 50])
next_states torch.float32 torch.Size([100, 51])
target_action torch.int64 torch.Size([50])


In [ ]:
# import matplotlib.pyplot as plt
# target_actions = [torch.argmax(sample['target_action']).item() for _ in range(10) for sample in dataset]
# bins = np.arange(0, 12)
# plt.hist(target_actions, bins-0.5)
# plt.xticks(bins[:-1])
# plt.show()

In [37]:
reward_model = Reward()
rewards = reward_model.offline(sample['states'], sample['actions'], sample['next_states'])

In [39]:
# torch.all(sample['states'][..., :-1].long() ^ sample['actions'][..., :-1] == sample['next_states'][..., :-1].long())

In [199]:
from dpt.loss import RKLLoss

In [211]:
d = 10
B = 1
S = 3
logits = torch.randn((B, S, d + 1))
targets = torch.randint(0, 2, (B, d + 1))

loss_fn = RKLLoss(label_smoothing=0)
loss_fn(logits, targets)

{'loss': tensor([[5.8750, 4.7763, 6.2020]])}

In [214]:
targets = torch.zeros((3, d + 1))
targets[0, 5] = 1
targets[[1]*3, [1, 5, 9]] = 1
targets[2] = 1

logits = torch.eye(d + 1).unsqueeze(0).repeat(3, 1, 1)
print(logits)
print(targets)
loss_fn(logits, targets)

tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        

{'loss': tensor([[10.3999, 10.3999, 10.3999, 10.3999, 10.3999,  8.5334, 10.3999, 10.3999,
          10.3999, 10.3999, 10.3999],
         [ 8.4865,  6.7684,  8.4865,  8.4865,  8.4865,  6.7684,  8.4865,  8.4865,
           8.4865,  6.7684,  8.4865],
         [ 0.0686,  0.0686,  0.0686,  0.0686,  0.0686,  0.0686,  0.0686,  0.0686,
           0.0686,  0.0686,  0.0686]])}

In [ ]:
d = 5
B = 1
S = 3
logits = torch.randn((B, S, d + 1))
targets = torch.randint(0, 2, (B, d + 1))#.unsqueeze(1)#.repeat(1, S, 1)

q = F.softmax(logits, dim=-1)
p = targets / targets.sum(dim=-1, keepdim=True)

loss_qq = (q * torch.log(q + 1e-6)).sum(-1)
loss_qp = (q * torch.log(p + 1e-6)).sum(-1)

print(loss_qq, loss_qp)
loss = (loss_qq - loss_qp)#.mean()
print(loss)

loss_fn = nn.CrossEntropyLoss()
loss_fn()

tensor([[-1.1794, -1.6698, -1.3440]]) tensor([[-13.2228, -13.0729, -13.1242]])
tensor([[12.0434, 11.4031, 11.7802]])


In [162]:
"""
predictions - [batch_size, seq_len + 1, action_dim]
targets     - [batch_size, action_dim]
"""

d = 10
B = 1
S = 3
predictions = torch.randint(0, d + 1, (B, S))
predictions = torch.eye(d + 1)[predictions]
targets = torch.randint(0, 2, (B, d + 1)).unsqueeze(1)#.repeat(1, S, 1)

accuracy = torch.any(predictions * targets, dim=-1).float().mean()
predictions = torch.argmax(predictions, dim=-1)

print('\ntargets')
print(targets[:, 0, :])
print('\ntarget indexes')
indexes = torch.arange(1, d + 2) * targets - 1
indexes[indexes < 0] = 3 * d # imax
print(indexes[:, 0, :])
print('\npredictions')
print(predictions)
print('\nresults')


dists = (indexes - predictions[..., None]).abs()
print(dists)
mae = dists.min(dim=-1).values.float().mean()
mae
# targets.where(targets == 1)
# mae = 


targets
tensor([[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1]])

target indexes
tensor([[30, 30, 30,  3,  4,  5,  6,  7, 30, 30, 10]])

predictions
tensor([[0, 5, 8]])

results
tensor([[[30, 30, 30,  3,  4,  5,  6,  7, 30, 30, 10],
         [25, 25, 25,  2,  1,  0,  1,  2, 25, 25,  5],
         [22, 22, 22,  5,  4,  3,  2,  1, 22, 22,  2]]])


tensor(1.3333)

In [41]:
sample = dataset[0]
print_sample(sample, rewards, 'point')

context
    00011001011100100111111111111010000011100110110010 8.041    ->  -> 10010011000001010110011110001010001011101011011101 -27.059  -> -2.443
    10010011000001010110011110001010001011101011011101 -27.059  ->  -> 00010000010110000010011010100100011111011110111011 -34.388  -> -1.618
    00010000010110000010011010100100011111011110111011 -34.388  ->  -> 00111110111100000010110010011111110000000100110111 -21.137  -> -1.578
    00111110111100000010110010011111110000000100110111 -21.137  ->  -> 00101110010010101110110001011110110011110101001011 -23.957  -> -2.398
    00101110010010101110110001011110110011110101001011 -23.957  ->  -> 00101100101011100101000111001000110010000110001001 1.647    -> -2.876
    00101100101011100101000111001000110010000110001001 1.647    ->  -> 10110101111010101001111101100010001101001111111111 -28.611  -> -2.822
    10110101111010101001111101100010001101001111111111 -28.611  ->  -> 01000001011101001101001000011011111001111011110011 -19.333  -> -3.478
    0

In [ ]:
collate_fn = partial(custom_collate_fn, problem_class=getattr(pbs, 'Problem'))
dataloader = DataLoader(dataset=dataset, batch_size=10, collate_fn=collate_fn)
batch = next(iter(dataloader))

for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.dtype, v.shape)
    else:
        print(k, type(v))

In [ ]:
# from train_dpt import ProblemDataModule, DPTSolver, L
from notebooks.utils import load_config

In [ ]:
config = load_config('../configs/config.yaml')
# config['batch_size'] = 3

# config['action'] = "point"
# # target_action: "gt"
# # ad_ratio: 0.0
# config['model_params']['action_dim'] = 50

datamodule = ProblemDataModule(config)
datamodule.setup()

In [ ]:
batch = next(iter(datamodule.test_dataloader()))

In [ ]:
model = DPTSolver(config)
model.configure_model()

In [ ]:
model.test_step(batch, 0)

In [ ]:
# rewards = model.reward_model.offline(
rewards = reward_model.offline(
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"]
)
rewards.shape

In [ ]:
outputs = model.model(
    query_state=batch["query_state"],
    states=batch["states"],
    actions=batch["actions"],
    next_states=batch["next_states"],
    rewards=rewards,
)
outputs.shape

In [ ]:
trainer = L.Trainer(
    logger=None,
    precision=config["precision"] if config["strategy"] != "fsdp" else None,
    max_epochs=config["max_epochs"],
    log_every_n_steps=config["log_every_n_steps"],
    default_root_dir=config["wandb_params"]["save_dir"],
    enable_model_summary=True,
    use_distributed_sampler=False,
    # strategy=config["strategy"]
)